IMPORTAÇÃO

In [6]:
from config_path import outros, relatorios
import pandas as pd
import warnings
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
warnings.simplefilter(action='ignore', category=UserWarning)

CARGA

In [7]:
col_corte = ['DATA', 'COD', 'DESCRIÇÃO','RUA', 'PREDIO', 'APTO', 'QTDE CORTE', 'VL CORTE','MOTIVO','FUNCIONARIO']
df_corte_geral = pd.read_excel(outros.ou_corte, sheet_name= "CORTE 2025", usecols= col_corte) 

col_estoque = ['Código', 'Estoque', 'Qtde Pedida','Bloqueado(Qt.Bloq.-Qt.Avaria)', 'Qt.Avaria']
df_estoque = pd.read_excel(outros.ou_86, usecols= col_estoque)

df_prod = pd.read_excel(relatorios.rel_96, usecols= ['DTULTENT', 'CODPROD'])


TRATAMENTO

In [8]:
filtro_data = df_corte_geral['DATA'].max() - pd.Timedelta(days= 10)
df_filtrado = df_corte_geral.loc[(df_corte_geral['DATA'] > filtro_data) & (df_corte_geral['MOTIVO'] != "DEPURAÇÃO DE PESO VARIÁVEL")].copy()
df_filtrado ['FUNCIONARIO'] = df_filtrado['FUNCIONARIO'].str.split().str[0] + " " + df_filtrado['FUNCIONARIO'].str.split().str[-1]

grupo = df_filtrado.groupby(["COD", "DESCRIÇÃO"]).agg(
    ULT_DATA = ("DATA", "max"),
    DIAS_CORTE = ("DATA", "nunique"),
    TOTAL_CORTE = ("QTDE CORTE", "sum"),
    VL_CORTE = ("VL CORTE", lambda x: round(x.sum(), 2))
).reset_index().astype({"COD": "int", "TOTAL_CORTE": "int"}).sort_values('DIAS_CORTE', ascending=False)

df_final = grupo.merge(df_estoque, left_on='COD', right_on= 'Código', how= 'left').drop(columns= 'Código')
df_final = df_final.merge(df_prod, left_on= 'COD', right_on= 'CODPROD', how= 'left').drop(columns= 'CODPROD')

df_final = df_final.fillna({'Estoque':0, "Qtde Pedida":0}).astype({"Estoque": "int", "Qtde Pedida": "int"})
df_final["BLOQUEIO"] = df_final["Bloqueado(Qt.Bloq.-Qt.Avaria)"].fillna(0).astype(int) + df_final["Qt.Avaria"].fillna(0).astype(int)
df_final = df_final.drop(columns= ['Bloqueado(Qt.Bloq.-Qt.Avaria)', 'Qt.Avaria'])
df_final ['DIFERENCA'] = df_final["Estoque"] - df_final["BLOQUEIO"] 


CONSULTA

In [9]:
caso_1 = df_final.loc[(df_final['DIFERENCA'] > 0) & (df_final['DIAS_CORTE'] > 1) & (df_final['ULT_DATA'] > df_final['DTULTENT'])].copy()
caso_1 = caso_1.drop(columns= ['DTULTENT','Qtde Pedida'])

total_corte = locale.currency(caso_1['VL_CORTE'].sum(), grouping=True)
qtde_corte = caso_1['DESCRIÇÃO'].count()
print(f"valor corte: {total_corte} || qtde: {qtde_corte}")
display(caso_1)

valor corte: R$ 2.992,16 || qtde: 6


,COD,DESCRIÇÃO,ULT_DATA,DIAS_CORTE,TOTAL_CORTE,VL_CORTE,Estoque,BLOQUEIO,DIFERENCA
3,469797,"SABAO PO AMACITEL 1,6KG SACHET TOQ POE",2025-12-05,3,6,362.47,6,4,2
6,442928,ACHOC TODDY REFORCADO SACHET 700GR,2025-12-08,2,2,333.70,1,0,1
8,447535,SAND HAV FEM SLIM L PTO 39/0,2025-12-11,2,5,136.65,5,4,1
11,306900,GUARD FS SANTEPEL GRANDE 33X30 50X1UN,2025-12-12,2,2,77.21,20,0,20
30,455562,SARDINHA ROBINSON CRUSOE 125GR OLEO,2025-12-10,2,7,1611.13,9,8,1
39,428813,DESINF PINHO BRIL 1LT FLORES LIMAO,2025-12-04,2,4,471.00,3,2,1


In [10]:
caso_2 = df_filtrado.loc[(df_filtrado['COD'] == 476146)]
caso_2 = caso_2[['DATA','COD', 'DESCRIÇÃO', 'RUA','QTDE CORTE', 'VL CORTE','MOTIVO','FUNCIONARIO']]


total_corte = locale.currency(caso_2['VL CORTE'].sum(), grouping=True)
qtde_corte = caso_2['DESCRIÇÃO'].count()
print(f"valor corte: {total_corte} || qtde: {qtde_corte}")
display(caso_2.sort_values(by='DATA', ascending= False))


valor corte: R$ 52,15 || qtde: 2


,DATA,COD,DESCRIÇÃO,RUA,QTDE CORTE,VL CORTE,MOTIVO,FUNCIONARIO
16939,2025-12-09,476146,BATATA PRINGLES 100GR NETFLIX BACON,32.0,1.0,9.20,QUANTIDADE INFERIOR DA OS,JANDIRA MOTA
16918,2025-12-08,476146,BATATA PRINGLES 100GR NETFLIX BACON,32.0,5.0,42.95,QUANTIDADE INFERIOR DA OS,TAYNARA ALMEIDA
